# LSTMを用いた連続音素認識

RNN(再帰ニューラルネット)は、記憶を持つニューラルネットです。現時刻の隠れ層の出力を、次の時刻の入力と共に隠れ層に入力することにより、状態を保持できる仕組みです。

LSTM (Long-Short Term Memory) はRNNの一種です。長期間の記憶を保持するために複雑な機構を有しています。今回は、時間的に順方向の記憶と逆方向の記憶を別個のユニットで保持する双方向LSTM (Bi-directional LSTM) を使います。

## ネットワーク構造の記述

今回のネットワークは、入力層から1番目の隠れ層が線形変換+ReLU, 次が双方向LSTM, 最後に出力層まで線形変換、という構造を持っています。`n_lstm_layers`は、双方向LSTMを何層重ねるかのパラメータです。

双方向LSTMの隠れユニットは順方向と逆方向の2組あるので、LSTMから出力層への結合を表す`l3`は $2H\times D$ 行列になります。

In [1]:
import chainer
import chainer.links as L
import chainer.functions as F

class RNN(chainer.Chain):

    def __init__(self, n_lstm_layers=1, n_mid_units=100, n_out=41, dropout=0.2):
        super(RNN, self).__init__()

        # パラメータを持つ層の登録
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            self.l2 = L.NStepBiLSTM(n_lstm_layers, n_mid_units, n_mid_units, dropout)
            self.l3 = L.Linear(n_mid_units * 2, n_out)

    def __call__(self, x):
        # データを受け取った際のforward計算を書く
        numframes = [X.shape[0] for X in x]
        split_point = np.cumsum(numframes)[:-1]
        x1 = F.concat(x, axis=0)
        h1 = F.relu(self.l1(x1))
        h1 = F.split_axis(h1, split_point, axis=0)
        hy, cy, ys = self.l2(None, None, h1)
        h2 = F.concat(ys, axis=0)
        return self.l3(h2)

/var/tmp/hiroki/conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


以下はfeed-forwardと同じ

In [2]:
from chainer.dataset import to_device
from chainer.dataset import concat_examples
import numpy as np

def converter(batch, device):
    # alternative to chainer.dataset.concat_examples
    
    Xs = [to_device(device, X) for X, _, __ in batch]
    ts = [to_device(device, t) for _, t, __ in batch]

    lab_batch = [lab.astype(np.int32) for _, __, lab in batch]
    labs = concat_examples(lab_batch, device, padding=0)
    
    return Xs, ts, labs

In [3]:
import numpy as np
from chainer.backends import cuda

def calculate_loss(net, batch, gpu_id=0, train_mode=True):
    numframes = [X.shape[0] for (X, t, lab) in batch]
    
    Xs, ts, _ = converter(batch, gpu_id) # 生ラベルは使わない

    with chainer.using_config('train', train_mode), \
         chainer.using_config('enable_backprop', train_mode):
        ys = net(Xs)
    
    ts = F.concat(ts, axis=0)
    ts = ts.reshape(ts.shape[0],)
    
    # ロスを計算
    loss = F.softmax_cross_entropy(ys, ts)

    # 精度を計算
    accuracy = F.accuracy(ys, ts)

    return loss, accuracy

In [4]:
import random
import numpy as np
import chainer

def reset_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    if gpu_id >= 0:
        chainer.cuda.cupy.random.seed(seed)

In [5]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

with open('phones') as f:
    phones = f.read().splitlines()
le = LabelEncoder()
le.fit(phones)
nsymbol = len(phones)

## ネットワークの学習
`MLP` のかわりに `RNN` でネットワークを作っているところだけが違います。

In [6]:
import numpy as np
from chainer import iterators
from chainer import optimizers
from chainer.backends import cuda
from chainer.cuda import to_cpu

gpu_id = 0 # CPUを用いる場合は、この値を-1にしてください

batchsize = 100
max_epoch = 400
n_mid_units = 200

xp = np
if gpu_id >= 0:
    xp = chainer.cuda.cupy

reset_seed(0) # 乱数の種をセット

train = np.load("MHT-train.npy") # train: 450 x (framelen x 26 , framelen, maxphonenum)
test = np.load("MHT-test.npy")

train_iter = iterators.SerialIterator(train, batchsize)

net = RNN(n_lstm_layers=1, n_mid_units=n_mid_units, n_out=nsymbol)

optimizer = optimizers.SGD(lr=0.1).setup(net)

if gpu_id >= 0:
    net.to_gpu(gpu_id)
    
while train_iter.epoch < max_epoch:

    # ---------- 学習の1イテレーション ----------
    train_batch = train_iter.next()

    loss, _ = calculate_loss(net, train_batch, gpu_id, train_mode = True)

    # 勾配の計算
    net.cleargrads()
    loss.backward()

    # バッチ単位で古い記憶を削除し、計算コストを削減する。
    loss.unchain_backward()

    # パラメータの更新
    optimizer.update()
    
    # --------------- ここまで ----------------

    # 1エポック終了ごとにValidationデータに対する予測精度を測って、
    # モデルの汎化性能が向上していることをチェックしよう
    if train_iter.is_new_epoch:  # 1 epochが終わったら
        # ロスの表示
        print('epoch:{:02d} train_loss:{:.04f} '.format(
            train_iter.epoch, float(to_cpu(loss.data))), end='')

        valid_losses = []
        valid_accuracies = []
        
        valid_loss, valid_accuracy = calculate_loss(net, test, gpu_id, train_mode=False)
        valid_losses.append(to_cpu(valid_loss.array))
        valid_accuracies.append(to_cpu(valid_accuracy.array))

        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(
            np.mean(valid_losses), np.mean(valid_accuracies)))

epoch:01 train_loss:2.4433 val_loss:2.3563 val_accuracy:0.4315
epoch:02 train_loss:2.1308 val_loss:2.0579 val_accuracy:0.4634
epoch:03 train_loss:1.8794 val_loss:1.8578 val_accuracy:0.4962
epoch:04 train_loss:1.7482 val_loss:1.7381 val_accuracy:0.5402
epoch:05 train_loss:1.6392 val_loss:1.6325 val_accuracy:0.5781
epoch:06 train_loss:1.5588 val_loss:1.5595 val_accuracy:0.6007
epoch:07 train_loss:1.5019 val_loss:1.4961 val_accuracy:0.5743
epoch:08 train_loss:1.4267 val_loss:1.4424 val_accuracy:0.5850
epoch:09 train_loss:1.3819 val_loss:1.3746 val_accuracy:0.6503
epoch:10 train_loss:1.3176 val_loss:1.3318 val_accuracy:0.6330
epoch:11 train_loss:1.3260 val_loss:1.3045 val_accuracy:0.6312
epoch:12 train_loss:1.2641 val_loss:1.2576 val_accuracy:0.6661
epoch:13 train_loss:1.2111 val_loss:1.2156 val_accuracy:0.6648
epoch:14 train_loss:1.1585 val_loss:1.2158 val_accuracy:0.6189
epoch:15 train_loss:1.1544 val_loss:1.1507 val_accuracy:0.6960
epoch:16 train_loss:1.1165 val_loss:1.1216 val_accuracy

epoch:131 train_loss:0.4174 val_loss:0.4775 val_accuracy:0.8557
epoch:132 train_loss:0.4162 val_loss:0.4762 val_accuracy:0.8565
epoch:133 train_loss:0.4179 val_loss:0.4739 val_accuracy:0.8586
epoch:134 train_loss:0.4291 val_loss:0.4751 val_accuracy:0.8584
epoch:135 train_loss:0.4213 val_loss:0.4706 val_accuracy:0.8588
epoch:136 train_loss:0.4216 val_loss:0.4668 val_accuracy:0.8600
epoch:137 train_loss:0.4244 val_loss:0.4730 val_accuracy:0.8580
epoch:138 train_loss:0.4167 val_loss:0.4711 val_accuracy:0.8595
epoch:139 train_loss:0.4427 val_loss:0.4865 val_accuracy:0.8537
epoch:140 train_loss:0.4126 val_loss:0.4745 val_accuracy:0.8574
epoch:141 train_loss:0.4192 val_loss:0.4681 val_accuracy:0.8603
epoch:142 train_loss:0.4056 val_loss:0.4605 val_accuracy:0.8642
epoch:143 train_loss:0.3885 val_loss:0.4661 val_accuracy:0.8597
epoch:144 train_loss:0.4118 val_loss:0.4673 val_accuracy:0.8602
epoch:145 train_loss:0.3861 val_loss:0.4598 val_accuracy:0.8630
epoch:146 train_loss:0.4057 val_loss:0.4

epoch:260 train_loss:0.3044 val_loss:0.3695 val_accuracy:0.8846
epoch:261 train_loss:0.3013 val_loss:0.3795 val_accuracy:0.8817
epoch:262 train_loss:0.3041 val_loss:0.3783 val_accuracy:0.8809
epoch:263 train_loss:0.2980 val_loss:0.3717 val_accuracy:0.8835
epoch:264 train_loss:0.2978 val_loss:0.3720 val_accuracy:0.8845
epoch:265 train_loss:0.2873 val_loss:0.3822 val_accuracy:0.8799
epoch:266 train_loss:0.2973 val_loss:0.3790 val_accuracy:0.8803
epoch:267 train_loss:0.2916 val_loss:0.3774 val_accuracy:0.8810
epoch:268 train_loss:0.2928 val_loss:0.3779 val_accuracy:0.8811
epoch:269 train_loss:0.3018 val_loss:0.3655 val_accuracy:0.8867
epoch:270 train_loss:0.3033 val_loss:0.3684 val_accuracy:0.8859
epoch:271 train_loss:0.2749 val_loss:0.3702 val_accuracy:0.8829
epoch:272 train_loss:0.2928 val_loss:0.3720 val_accuracy:0.8815
epoch:273 train_loss:0.2885 val_loss:0.3735 val_accuracy:0.8846
epoch:274 train_loss:0.2852 val_loss:0.3714 val_accuracy:0.8863
epoch:275 train_loss:0.2829 val_loss:0.3

epoch:389 train_loss:0.2312 val_loss:0.3286 val_accuracy:0.8954
epoch:390 train_loss:0.2401 val_loss:0.3315 val_accuracy:0.8946
epoch:391 train_loss:0.2337 val_loss:0.3218 val_accuracy:0.8991
epoch:392 train_loss:0.2419 val_loss:0.3265 val_accuracy:0.8995
epoch:393 train_loss:0.2374 val_loss:0.3259 val_accuracy:0.8966
epoch:394 train_loss:0.2351 val_loss:0.3235 val_accuracy:0.8973
epoch:395 train_loss:0.2386 val_loss:0.3409 val_accuracy:0.8937
epoch:396 train_loss:0.2508 val_loss:0.3570 val_accuracy:0.8872
epoch:397 train_loss:0.2379 val_loss:0.3209 val_accuracy:0.8986
epoch:398 train_loss:0.2330 val_loss:0.3240 val_accuracy:0.8978
epoch:399 train_loss:0.2359 val_loss:0.3266 val_accuracy:0.8966
epoch:400 train_loss:0.2420 val_loss:0.3268 val_accuracy:0.8956


フレームごとの音素認識率は90%程度と、feed-forwardに比べて改善していることがわかります。

## 音素認識結果の観察

In [7]:
test_utterance_number = 0

if gpu_id >= 0:
    net.to_gpu(gpu_id)

Xs, ts, _ = converter(test, gpu_id)

# テストデータを1つ取り出します
X_test = Xs[test_utterance_number]
t_test = ts[test_utterance_number]

with chainer.using_config('train', False), \
     chainer.using_config('enable_backprop', False):
    y_test = net([X_test])

# Variable形式で出てくるので中身を取り出す
y_test = y_test.array

# 結果をCPUに送る
y_test = to_cpu(y_test)
t_test = to_cpu(t_test)

# 予測確率の最大値のインデックスを見る
pred_label = y_test.argmax(axis=1)

In [8]:
le.inverse_transform(t_test)

/var/tmp/hiroki/conda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch',
       'ch', 'ch', 'ch', 'ch', 'ch', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'i', 'i', 'i', 'i', 's', 's', 's', 's', 's', 's', 's', 's', 's',
       's', 'a', 'a', 'a', 'a', 'a', 'a', 'n', 'n', 'n', 'n', 'a', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'a', 'u', 'u', 'u', 'u', 'u', 'u',
       'u', 'u', 'u', 'n', 'n', 'n', 'n', 'n', 'a', 'a', 'a', 'a', 'a',
       'a', 'a', 'a', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'g', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'y', 'y', 'y', 'y', 'y', 'y', 'y', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'a', 'n', 'n', 'n', 'n', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',

In [9]:
le.inverse_transform(pred_label)

/var/tmp/hiroki/conda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil', 'sil',
       'ch', 'sil', 'sil', 'sil', 'ch', 'ch', 'ch', 'ch', 'ch', 'ch',
       'ch', 'ch', 'ch', 'ch', 'ch', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 's', 's', 's', 's', 's', 's', 's',
       's', 's', 'a', 'a', 'a', 'a', 'a', 'n', 'n', 'n', 'n', 'n', 'n',
       'a', 'a', 'a', 'a', 'a', 'e', 'e', 'u', 'u', 'u', 'u', 'u', 'u',
       'u', 'u', 'u', 'n', 'n', 'n', 'n', 'n', 'n', 'a', 'a', 'a', 'a',
       'a', 'a', 'a', 'e', 'e', 'g', 'g', 'n', 'n', 'g', 'g', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'r',
       'a', 'a', 'a', 'a', 'a', 'a', 'a', 'r', 'n', 'n', 'n', 'n', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i',
       'i', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau',
       'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau

前後関係が考慮され、音素が系列として認識されています。記憶を持つネットワークにより、日本語の音素列としてありそうもないものは出力されにくくなっていることがわかります。